In [1]:
import pandas as pd

df=pd.read_table("data_files/class.tsv")
print(df.head())

  Gender  Age   Height   Weight
0      F   11  130.302  22.9068
1      M   11  146.050  38.5560
2      M   12  145.542  37.6488
3      F   12  151.892  38.3292
4      M   12  149.860  45.1332


In [2]:
# select specific column
df['Weight']

0     22.9068
1     38.5560
2     37.6488
3     38.3292
4     45.1332
5     34.9272
6     58.0608
7     38.1024
8     44.4528
9     38.1024
10    46.4940
11    46.4940
12    51.0300
13    40.8240
14    51.0300
15    50.8032
16    60.3288
17    50.8032
18    68.0400
Name: Weight, dtype: float64

In [3]:
# select rows first 4 rows and all columns
print(df.loc[0:3,:])

  Gender  Age   Height   Weight
0      F   11  130.302  22.9068
1      M   11  146.050  38.5560
2      M   12  145.542  37.6488
3      F   12  151.892  38.3292


In [4]:
# print first 6 rows and the columns gender and height
print(df.loc[0:5,['Gender', 'Height']])

  Gender   Height
0      F  130.302
1      M  146.050
2      M  145.542
3      F  151.892
4      M  149.860
5      F  143.002


In [5]:
df.loc[0:1, ['Gender', 'Age']]

,Gender,Age
0,F,11
1,M,11


In [6]:
# create a mask and select age above 14
maskAge=df['Age']>14
print(maskAge)
print(maskAge.value_counts())

0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14     True
15     True
16     True
17     True
18     True
Name: Age, dtype: bool
Age
False    14
True      5
Name: count, dtype: int64


In [7]:
# apply mask to df
df.loc[maskAge, ['Age', 'Height', 'Weight']]

,Age,Height,Weight
14,15,158.75,51.0300
15,15,168.91,50.8032
16,15,170.18,60.3288
17,15,168.91,50.8032
18,16,182.88,68.0400


In [8]:
mask = (df['Age']>14) & (df['Gender']=='F')
print(df.loc[mask,])

   Gender  Age  Height   Weight
14      F   15  158.75  51.0300
15      F   15  168.91  50.8032


In [9]:
mask2= (df['Gender']=='M') & (df['Weight']<40)
print(df.loc[mask2,])

  Gender  Age   Height   Weight
1      M   11  146.050  38.5560
2      M   12  145.542  37.6488
9      M   13  158.750  38.1024


In [10]:
df_heart=pd.read_table("data_files/heartData_simplified.csv", sep=",", header=0)
mask_heart=df_heart['chol']<200
print(df_heart.loc[mask_heart,])
print(mask_heart.value_counts())

     age     sex  chol  thalach  oldpeak  ca        thal target
5     57    male   192      148      0.4   0      defect     no
8     52    male   199      162      0.5   0  reversible     no
9     57    male   168      174      1.6   0      normal     no
24    40    male   199      178      1.4   0  reversible     no
27    51    male   175      123      0.6   0      normal     no
29    53    male   197      152      1.2   0      normal     no
30    41  female   198      168      0.0   1      normal     no
31    65    male   177      140      0.4   0  reversible     no
35    46  female   177      160      1.4   0      normal     no
52    44  female   141      175      0.6   0      normal     no
57    34    male   182      174      0.0   0      normal     no
61    52    male   186      190      0.0   0      defect     no
64    35  female   183      182      1.4   0      normal     no
86    46    male   197      156      0.0   0  reversible     no
92    45  female   160      138      0.0

In [11]:
# print mean of age from the previous mask
print("mean age:")
print(df_heart.loc[mask_heart,'age'].mean())


mean age:
51.979166666666664


Many other statistical methods, such as PCA, work best when the data has been centered and scaled (meaning that we removed its mean and divided by its standard deviation).

In [12]:
dat=pd.read_table("data_files/GSE41558_series_matrix.reduced.tsv", index_col=0)
print(dat.head())
print(dat.shape)

              Heart_WT_1  Heart_WT_2  Heart_WT_3  Heart_WT_4  Heart_KO_1  \
gene_id                                                                    
1415670_at     1214.4470   1182.4640   1206.2260   1196.0300   1174.6180   
1415671_at     3490.0980   2882.7840   2650.0330   2934.8610   2723.9760   
1415672_at     4510.3690   4292.0570   4071.0570   4275.2760   4127.9410   
1415673_at      598.8334    385.0178    458.4872    514.2919    584.0671   
1415674_a_at   1400.3250   1328.2950   1416.9230   1388.4180   1459.9560   

              Heart_KO_2  Heart_KO_3  Heart_KO_4  
gene_id                                           
1415670_at     1184.4580    985.5503   1214.5400  
1415671_at     2823.2600   2721.8840   2790.8340  
1415672_at     4045.9900   4553.7360   4358.6350  
1415673_at      544.3807    569.1154    323.8668  
1415674_a_at   1462.9640   1237.2440   1797.9060  
(349, 8)


In [13]:
# applying a custom function
def expressionFilter(rowVal, minExpr=10, minSample=5):
    mask=rowVal>=minExpr # creates a mask to check for minexpression
    return sum(mask)>=minSample # return True if at least minsample samples satidfy the filter

In [14]:
expressionFilterMask=dat.apply(expressionFilter, axis=1) # axis=1, means it is applied for each row. axis=0 would apply it to each column
print(expressionFilterMask.value_counts())

True     333
False     16
Name: count, dtype: int64


In [15]:
# next apply a custom function to each column
import numpy as np

def calculate_mean_log(iterable):
    return np.average(np.log(iterable))

#apply this along columns
meanlogexpression=dat.apply(calculate_mean_log, axis=0)
print(meanlogexpression)

Heart_WT_1    6.955806
Heart_WT_2    6.966364
Heart_WT_3    6.933558
Heart_WT_4    6.958315
Heart_KO_1    6.986187
Heart_KO_2    6.928704
Heart_KO_3    6.937114
Heart_KO_4    6.961720
dtype: float64


In [16]:
dat['Heart_WT_1'].head()

gene_id
1415670_at      1214.4470
1415671_at      3490.0980
1415672_at      4510.3690
1415673_at       598.8334
1415674_a_at    1400.3250
Name: Heart_WT_1, dtype: float64

In [17]:
print(dat['Heart_WT_1'].head()/1000)

gene_id
1415670_at      1.214447
1415671_at      3.490098
1415672_at      4.510369
1415673_at      0.598833
1415674_a_at    1.400325
Name: Heart_WT_1, dtype: float64


perform operations on several columns

In [18]:
avg=(dat['Heart_WT_1'] + dat['Heart_WT_2'])/2
print(avg)
print( avg.mean() )

gene_id
1415670_at      1198.45550
1415671_at      3186.44100
1415672_at      4401.21300
1415673_at       491.92560
1415674_a_at    1364.31000
                   ...    
1416014_at      1254.45500
1416015_s_at    2057.40250
1416016_at       348.10085
1416017_at       579.67175
1416018_at      1330.89700
Length: 349, dtype: float64
4676.56708867192


In [ ]:
meanExpressionWT= (dat['Heart_WT_1']+dat['Heart_WT_2']+dat['Heart_WT_3']+dat['Heart_WT_4'])/4
print(meanExpressionWT)
print(meanExpressionWT.mean())

In [ ]:
# add a new column to your df. just like adding a new key to a dictionary
dat['Heart_WT_avg']=meanExpressionWT
print(dat.head())

In [ ]:
# delete a column using drop(). axis=1, drops columns and axis=0, drops rows
# inplace=True: makes the drop function modify the DataFrame itself rather than return a copy of it without the column.

dat.drop('Heart_WT_avg', axis=1, inplace=True)
print(dat.head())

In [ ]:
chol_gl=df_heart['chol']/100
df_heart['chol_gl']=chol_gl
print(df_heart.head())

print(df_heart.loc[:,['chol','chol_gl']].describe())

In [21]:
def expressionFilter(iterable , minExpr=10 , minSample=4):
    ''' returns True is at least <minSample> have a value of at least <minExpr> in <iterable>'''
    mask = iterable>=minExpr  # creating a mask to check for mininmum expression
    return sum(mask)>=minSample # Return True if at least minSample samples satisfy the filter

# apply the function along rows
expressionFilterMask =  dat.apply(expressionFilter , axis=1)

print('expression filter for lowly expressed genes')
print( expressionFilterMask.value_counts() ) # we print the count of the custom mask we just created


expression filter for lowly expressed genes
True     335
False     14
Name: count, dtype: int64


In [23]:
dat_filtered=dat.loc[expressionFilterMask,:]
print(dat_filtered.head())

              Heart_WT_1  Heart_WT_2  Heart_WT_3  Heart_WT_4  Heart_KO_1  \
gene_id                                                                    
1415670_at     1214.4470   1182.4640   1206.2260   1196.0300   1174.6180   
1415671_at     3490.0980   2882.7840   2650.0330   2934.8610   2723.9760   
1415672_at     4510.3690   4292.0570   4071.0570   4275.2760   4127.9410   
1415673_at      598.8334    385.0178    458.4872    514.2919    584.0671   
1415674_a_at   1400.3250   1328.2950   1416.9230   1388.4180   1459.9560   

              Heart_KO_2  Heart_KO_3  Heart_KO_4  
gene_id                                           
1415670_at     1184.4580    985.5503   1214.5400  
1415671_at     2823.2600   2721.8840   2790.8340  
1415672_at     4045.9900   4553.7360   4358.6350  
1415673_at      544.3807    569.1154    323.8668  
1415674_a_at   1462.9640   1237.2440   1797.9060  


In [24]:
# calculate means of each column
col_means=dat_filtered.mean(axis=0)
print(col_means.head())

Heart_WT_1    4836.696304
Heart_WT_2    4906.362128
Heart_WT_3    4609.446266
Heart_WT_4    4371.035544
Heart_KO_1    5245.642994
dtype: float64


In [25]:
# calculate the stdev of each column
stdev_col=dat_filtered.std(axis=0)
print(stdev_col.head())

Heart_WT_1     9922.790015
Heart_WT_2    10528.565141
Heart_WT_3     9311.387936
Heart_WT_4     8320.238000
Heart_KO_1    11430.048742
dtype: float64


In [26]:
# centering and scaling. subtract the mean and divide by the stdev
dat_center_scale=(dat_filtered - col_means)/stdev_col
print(dat_center_scale.head())

              Heart_WT_1  Heart_WT_2  Heart_WT_3  Heart_WT_4  Heart_KO_1  \
gene_id                                                                    
1415670_at     -0.365043   -0.353695   -0.365490   -0.381600   -0.356169   
1415671_at     -0.135708   -0.192199   -0.210432   -0.172612   -0.220617   
1415672_at     -0.032887   -0.058347   -0.057821   -0.011509   -0.097786   
1415673_at     -0.427084   -0.429436   -0.445794   -0.463538   -0.407835   
1415674_a_at   -0.346311   -0.339844   -0.342862   -0.358477   -0.331205   

              Heart_KO_2  Heart_KO_3  Heart_KO_4  
gene_id                                           
1415670_at     -0.369123   -0.372349   -0.370651  
1415671_at     -0.202230   -0.213680   -0.206529  
1415672_at     -0.077708   -0.046283   -0.043291  
1415673_at     -0.434308   -0.410403   -0.463387  
1415674_a_at   -0.340761   -0.349349   -0.309912  
